In [1]:
import cirq
import numpy as np
from functools import reduce

class C3SXGate(cirq.Gate):
    def __init__(self):
        super(C3SXGate, self)

    def _num_qubits_(self):
        return 4

    def _decompose_(self, qubits):
        pi = np.pi;
        a, b, c, d = qubits
        
        yield cirq.H(d)
        yield CU1Gate(pi/8)(a, d)
        yield cirq.H(d)
        yield cirq.CX(a, b)
        yield cirq.H(d)
        yield CU1Gate(-pi/8)(b, d)
        yield cirq.H(d)
        yield cirq.CX(a, b)
        yield cirq.H(d)
        yield CU1Gate(pi/8)(b, d)
        yield cirq.H(d)
        yield cirq.CX(b, c)
        yield cirq.H(d)
        yield CU1Gate(-pi/8)(c, d) 
        yield cirq.H(d)
        yield cirq.CX(a,c)
        yield cirq.H(d)
        yield CU1Gate(pi/8)(c, d)
        yield cirq.H(d)
        yield cirq.CX(b, c)
        yield cirq.H(d)
        yield CU1Gate(-pi/8)(c, d)
        yield cirq.H(d)
        yield cirq.CX(a, c)
        yield cirq.H(d)
        yield CU1Gate(pi/8)(c, d)
        yield cirq.H(d)

    def _circuit_diagram_info_(self, args):
        return "@", '@', '@', 'C3SX'    

class U1Gate(cirq.Gate):
    def __init__(self, lam):
        super(U1Gate, self)
        self.lam = lam

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        lam = float(self.lam)
        return np.array([[1, 0], [0, np.exp(1j * lam)]])

    def _circuit_diagram_info_(self, args):
        return f"U1({self.lam:.2f})"

class CU1Gate(cirq.Gate):
    def __init__(self, eith):
        super(CU1Gate, self)
        self.eith = eith

    def _num_qubits_(self):
        return 2
    
    def _decompose_(self, qubits):
        a, b = qubits
        angle = self.eith;

        yield U1Gate(angle/2)(a)
        yield cirq.CX(a, b)
        yield U1Gate(-angle/2)(b)
        yield cirq.CX(a, b)
        yield U1Gate(angle/2)(b)

    def _circuit_diagram_info_(self, args):
        return f"CU1({self.eith:.2f})", "CU1"

q = [cirq.NamedQubit('q' + str(i)) for i in range(4)]

circuit = cirq.Circuit(
    # cirq.H(q[0]),
    # cirq.H(q[3]),
    C3SXGate()(q[0], q[3], q[1], q[2]),
    cirq.measure(q[0], key='cr0'),
    cirq.measure(q[1], key='cr1'),
    cirq.measure(q[2], key='cr2'),
    cirq.measure(q[3], key='cr3'),
)

print(circuit)
# print(cirq.qasm(circuit))
print(cirq.unitary(circuit).__repr__())
# print(circuit.unitary().__repr__())

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1024)
result_dict = dict(result.multi_measurement_histogram(keys=reversed(['cr0', 'cr1', 'cr2', 'cr3'])))
# keys = list(map(lambda arr: reduce(lambda x, y: str(x) + str(y), arr[::-1]), result_dict.keys()))
# counts = dict(zip(keys,[value for value in result_dict.values()]))
print(result_dict)
# counts
# result_dict

q0: ───@──────M('cr0')───
       │
q1: ───@──────M('cr1')───
       │
q2: ───C3SX───M('cr2')───
       │
q3: ───@──────M('cr3')───
array([[ 1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         3.14018492e-16+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [ 0.00000000e+00+0.00000000e+00j,  1.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  3.14018492e-16-1.96261557e-17j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+